# Prepare for your input

In [75]:
import pandas as pd
import json
df = pd.read_csv(r"C:\Users\user\Desktop\Courses\2024Winter\COMP313\Code\output_landmarks.csv", header=None, names=["index", "landmark", "target"])


def convert_to_json(string):
    try:
        string = string.replace("'", "\"")
        string = string.replace("\"\"\"", "\"'\"")
        return json.loads(string)
    except ValueError:
        print(string)
        return None


df['landmark'] = df['landmark'].apply(json.loads)


input_landmarks =df['landmark'].iloc[0]


for i in range(1):
    # Access the ith element in 'landmarks'
    landmarks_seq = df['landmark'].iloc[i]
    # Calculate the number of frames
    num_frames = len(landmarks_seq)
    # For lists, manually determine the shape of the first frame (if available)
    first_frame_shape = (len(landmarks_seq[0]), len(landmarks_seq[0][0])) if num_frames > 0 and len(landmarks_seq[0]) > 0 else "No frames"
    print(f"Sequence {i} has {num_frames} frames; First frame shape: {first_frame_shape}")




#print(input_landmarks.shape)

Sequence 0 has 19 frames; First frame shape: (21, 3)


# Step 1: Load the Trained Model

In [86]:
from tensorflow.keras.models import load_model

# Load the model from the file
loaded_model = load_model('model_sign_2d.h5')

# Step 2: Prepare Your New Input Data

# Step 3: Make Predictions with the Model

# application

In [87]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

def normalize_landmarks(landmarks_seq):
    # This will hold the normalized sequences
    normalized_seq = []
    
    for matrix in landmarks_seq:
        # Ensure the matrix is not empty to avoid errors during scaling
        if matrix:  # Checks if the matrix is not empty
            matrix_np = np.array(matrix)
            # Normalize the [21x3] matrix
            # Note: scaler.fit_transform expects a 2D array, [samples, features]
            # Reshape is used to ensure the matrix is 2D and to revert it back to original shape after scaling
            scaled_matrix = scaler.fit_transform(matrix_np.reshape(-1, 3)).reshape(21, 3)
            normalized_seq.append(scaled_matrix)
        else:
            # Handle empty matrices by appending an array of zeros or another placeholder
            # This keeps the temporal structure consistent
            normalized_seq.append(np.zeros((21, 3)))
    
    return normalized_seq

input_landmarks =df['landmark'].iloc[0]
normaized_landmarks =normalize_landmarks(input_landmarks)

In [88]:
# Step 2: Pad the scaled landmark data
max_frames = 125  # Target number of frames for each sequence

def pad_sequence(sequence):
    # Determine the number of frames to pad
    num_frames_to_pad = max_frames - len(sequence)
    
    # Create frames of zeros to be appended
    # Each frame is a (21, 3) array of zeros
    padding_frames = [np.zeros((21, 3)) for _ in range(num_frames_to_pad)]
    
    # Append the padding frames to the sequence
    padded_sequence = sequence + padding_frames
    
    return padded_sequence


processed_landmarks =pad_sequence(normaized_landmarks)


In [89]:
X_test = np.expand_dims(processed_landmarks, axis=0)

In [90]:
import numpy as np

def predict_sequence(model, X_test, sequence_length=17, num_decoder_tokens=27, start_token_index=0):
    """
    Predict sequences without relying on y_test, using a start token and iterative predictions.
    """
    decoder_input_test = np.zeros((len(X_test), sequence_length, num_decoder_tokens))
    decoder_input_test[:, 0, start_token_index] = 1  # Start with the start token
    
    for i in range(1, sequence_length):
        predicted_sequences = model.predict([X_test, decoder_input_test])
        next_token = np.argmax(predicted_sequences[:, i-1, :], axis=-1)
        decoder_input_test = np.zeros_like(decoder_input_test)
        for j in range(i + 1):
            decoder_input_test[np.arange(len(X_test)), j, next_token if j == i else start_token_index] = 1

    return predicted_sequences

def trim_and_analyze_predicted_sequences(predicted_sequences, pad_token=0):
    """
    Trim the predicted sequences by removing trailing pad tokens and analyze them.
    """
    predicted_labels = np.argmax(predicted_sequences, axis=-1)  # Convert to label indices
    
    def trim_sequences(sequences, pad_token):
        trimmed_sequences = []
        for seq in sequences:
            last_valid_index = -1
            for idx, token in enumerate(seq):
                if token != pad_token:
                    last_valid_index = idx
            trimmed_seq = seq[:last_valid_index+1] if last_valid_index != -1 else []
            trimmed_sequences.append(trimmed_seq)
        return trimmed_sequences
    
    predicted_labels_trimmed = trim_sequences(predicted_labels, pad_token)
    
    return predicted_labels_trimmed

# Assuming `model` and `X_test` are already defined and prepared
predicted_sequences = predict_sequence(model, X_test)
predicted_labels_trimmed = trim_and_analyze_predicted_sequences(predicted_sequences)

num_sequences_to_display = 10
for i, seq in enumerate(predicted_labels_trimmed[:num_sequences_to_display]):
    print(f"Sequence {i+1}: {seq}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Sequence 1: [4 3]


In [96]:
import joblib

# Load the label encoder from a file
label_encoder = joblib.load('label_encoder.joblib')

# Function to decode a single sequence of label indices into labels
def decode_sequence(sequence, label_encoder, pad_token=None):
    if pad_token is not None:
        # Filter out padding tokens if your pad_token is not a valid label index
        valid_indices = [index for index in sequence if index != pad_token]
    else:
        valid_indices = sequence
    # Decode the sequence of indices to labels
    labels = label_encoder.inverse_transform(valid_indices)
    return labels

# Decode all trimmed predicted sequences
decoded_sequences = [decode_sequence(seq, label_encoder) for seq in predicted_labels_trimmed]

# Display the decoded sequences
for i, decoded_seq in enumerate(decoded_sequences[:num_sequences_to_display], 1):
    print(f"Decoded Sequence {i}: {' '.join(decoded_seq)}")


Decoded Sequence 1: d c
